<style>
    body, * {
        direction: rtl !important;
        text-align: right !important;
    }
</style>
# 📚 PySpark - when() ו-otherwise() - תנאים במסגרת נתונים
[PySpark When Otherwise](https://sparkbyexamples.com/pyspark/pyspark-when-otherwise/)
## מדריך מקיף לשימוש בתנאים מותנים ב-PySpark

## 📖 מבוא

ב-PySpark, הפונקציות `when()` ו-`otherwise()` דומות לביטויי SQL `CASE WHEN`. הן מאפשרות לנו לבדוק מספר תנאים ברצף ולהחזיר ערך כאשר התנאי הראשון מתקיים.

**שימושים נפוצים:**
- בדיקת תנאים מרובים על עמודות DataFrame
- יצירת עמודות נגזרות על בסיס לוגיקה מותנית  
- הרחבת פונקציונליות דומה ל-`SWITCH` ו-`IF-THEN-ELSE`

הפונקציות פועלות באופן דומה לפקודות `"switch"` ו-`"if then else"` בשפות תכנות אחרות.

## 📋 תוכן עניינים

1. [שימוש ב-when() otherwise() על DataFrame](#1)
2. [PySpark SQL Case When על DataFrame](#2)
   - 2.1 [שימוש ב-Case When עם withColumn() ו-select()](#2.1)
   - 2.2 [שימוש ב-Case When עם ביטויי SQL](#2.2)
   - 2.3 [תנאים מרובים עם אופרטורים & ו-|](#2.3)
3. [דוגמה מלאה](#3)

## 🎯 יצירת DataFrame לדוגמה

ראשית, ניצור DataFrame פשוט עם מידע על אנשים כדי להדגים את השימוש בפונקציות.

In [ ]:
from pyspark.sql import SparkSession

# יצירת SparkSession
spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

# נתונים לדוגמה
data = [("James", "M", 60000), 
        ("Michael", "M", 70000),
        ("Robert", None, 400000), 
        ("Maria", "F", 500000),
        ("Jen", "", None)]

# הגדרת שמות העמודות
columns = ["name", "gender", "salary"]

# יצירת DataFrame
df = spark.createDataFrame(data=data, schema=columns)
df.show()

---

<a id="1"></a>
## 1️⃣ שימוש ב-when() otherwise() על PySpark DataFrame

הפונקציה `when()` היא פונקציית SQL ב-PySpark. כדי להשתמש בה, יש לייבא אותה מהמודול `pyspark.sql.functions`.

### 🔍 איך זה עובד?

הפונקציה מחזירה עמודה מסוג `Column`, ו-`otherwise()` היא פונקציית עזר של עמודה. אם `otherwise()` לא מוגדרת, הערך המוחזר יהיה `None` (NULL).

**תבנית השימוש:**
```python
when(condition).otherwise(default)
```

### 📝 פרמטרים:
- **condition** - התנאי לבדיקה
- **value** - הערך שיוחזר אם התנאי מתקיים

הקוד הבא מחליף את הערכים בעמודת המגדר (gender) בערך נגזר חדש. כאשר התנאים לא מתקיימים, אנו מקצים "Unknown" כערך, לשימוש עבור null או ריק.

In [ ]:
from pyspark.sql.functions import when

df2 = df.withColumn("new_gender", 
                    when(df.gender == "M", "Male")
                    .when(df.gender == "F", "Female")
                    .when(df.gender.isNull(), "")
                    .otherwise(df.gender))

df2.show()

### 📊 הסבר על הפלט:
- **James** ו-**Michael** - מגדר "M" הומר ל-"Male"
- **Maria** - מגדר "F" הומר ל-"Female"  
- **Robert** - null נשאר ריק
- **Jen** - מחרוזת ריקה עברה ל-otherwise והשארה כמות שהיא

### 🎯 שימוש עם select()

ניתן להשיג את אותה התוצאה גם עם `select()`.

In [ ]:
df2 = df.select("*", 
                 when(df.gender == "M", "Male")
                 .when(df.gender == "F", "Female")
                 .when(df.gender.isNull(), "")
                 .otherwise(df.gender).alias("new_gender"))

df2.show()

---

<a id="2"></a>
## 2️⃣ PySpark SQL Case When על DataFrame

אם יש לך רקע ב-SQL, אולי אתה מכיר את פקודת **Case When** המשמשת לביצוע רצף תנאים ומחזירה ערך כאשר התנאי הראשון מתקיים, דומה ל-SWITCH ו-IF THEN ELSE.

### 📐 תחביר של SQL CASE WHEN ELSE END

```sql
CASE
    WHEN condition1 THEN result_value1
    WHEN condition2 THEN result_value2
    -----
    ELSE result
END;
```

### הסבר על המרכיבים:
- ✅ **CASE** - התחלת הביטוי
- ✅ **Clause WHEN** - מקבלת תנאי, אם התנאי אמיתי מחזירה ערך מ-**THEN**
- ✅ **IF** - אם התנאי שקרי, ממשיכה לתנאי הבא וכן הלאה
- ✅ אם אף אחד מהתנאים לא מתקיים, מחזירה ערך מה-**ELSE**
- ✅ **END** - סוף הביטוי

---

<a id="2.1"></a>
### 2.1 שימוש ב-Case When Else עם withColumn() ו-select()

הדוגמה הבאה משתמשת ב-`PySpark SQL expr()` כדי לבטא ביטויים דמויי SQL.

In [ ]:
from pyspark.sql.functions import expr, col

# שימוש עם withColumn()
df3 = df.withColumn("new_gender", 
                    expr("CASE WHEN gender = 'M' THEN 'Male' " +
                         "WHEN gender = 'F' THEN 'Female' " +
                         "WHEN gender IS NULL THEN '' " +
                         "ELSE gender END"))
df3.show(truncate=False)

זה יניב את אותה התוצאה כמו בדוגמה הקודמת.

In [ ]:
# שימוש עם select()
df4 = df.select(col("*"), 
                expr("CASE WHEN gender = 'M' THEN 'Male' " +
                     "WHEN gender = 'F' THEN 'Female' " + 
                     "WHEN gender IS NULL THEN '' " +
                     "ELSE gender END").alias("new_gender"))

df4.show(truncate=False)

---

<a id="2.2"></a>
### 2.2 שימוש ב-Case When עם ביטוי SQL

ניתן להשתמש גם ב-Case When עם פקודת SQL לאחר יצירת temporary view. זה מחזיר פלט דומה לדוגמה למעלה.

In [ ]:
# יצירת Temporary View
df.createOrReplaceTempView("EMP")

# הרצת שאילתת SQL
spark.sql("SELECT name, CASE WHEN gender = 'M' THEN 'Male' " +
          "WHEN gender = 'F' THEN 'Female' " +
          "WHEN gender IS NULL THEN '' " +
          "ELSE gender END as new_gender FROM EMP").show()

---

<a id="2.3"></a>
### 2.3 תנאים מרובים עם אופרטורים & ו-| (AND ו-OR)

לעיתים קרובות אנו צריכים לבדוק תנאים מרובים. להלן דוגמה לשימוש ב-PySpark When Otherwise עם תנאים מרובים באמצעות אופרטורי `&` (AND) או `|` (OR). 

כדי להסביר זאת, אשתמש במערך נתונים חדש כדי שיהיה פשוט יותר.

In [ ]:
# יצירת DataFrame חדש לדוגמה
data = [("a", "1"),
        ("b", "4"),
        ("c", "3"),
        ("d", "2")]
columns = ["code", "cnt"]
df = spark.createDataFrame(data=data, schema=columns)
df.show()

In [ ]:
df5 = df.withColumn("new_column",
                    when((col("code") == "a") | (col("code") == "d"), "A")
                    .when((col("code") == "b") & (col("cnt") == "4"), "B")
                    .otherwise("A1"))

df5.show()

---

<a id="3"></a>
## 3️⃣ דוגמה מלאה - PySpark When Otherwise

להלן קוד Python מלא עם כל הדוגמאות שהוצגו במדריך זה:

In [ ]:
from pyspark.sql import SparkSession

# יצירת Spark Session
spark = SparkSession.builder.appName("SparkByExamples.com").getOrCreate()

# הכנת נתונים
data = [("James", "M", 60000), 
        ("Michael", "M", 70000),
        ("Robert", None, 400000), 
        ("Maria", "F", 500000),
        ("Jen", "", None)]

columns = ["name", "gender", "salary"]
df = spark.createDataFrame(data=data, schema=columns)
df.show()

# ====================================
# שימוש ב-when() otherwise()
# ====================================
from pyspark.sql.functions import when, col

df2 = df.withColumn("new_gender", 
                    when(df.gender == "M", "Male")
                    .when(df.gender == "F", "Female")
                    .when(df.gender.isNull(), "")
                    .otherwise(df.gender))
df2.show()

# עם select()
df2 = df.select("*", 
                when(df.gender == "M", "Male")
                .when(df.gender == "F", "Female")
                .when(df.gender.isNull(), "")
                .otherwise(df.gender).alias("new_gender"))
df2.show()

# ====================================
# שימוש ב-SQL Case When
# ====================================
from pyspark.sql.functions import expr

# עם withColumn()
df3 = df.withColumn("new_gender", 
                    expr("CASE WHEN gender = 'M' THEN 'Male' " +
                         "WHEN gender = 'F' THEN 'Female' " +
                         "WHEN gender IS NULL THEN '' " +
                         "ELSE gender END"))
df3.show()

# עם select()
df4 = df.select(col("*"), 
                expr("CASE WHEN gender = 'M' THEN 'Male' " +
                     "WHEN gender = 'F' THEN 'Female' " +
                     "WHEN gender IS NULL THEN '' " +
                     "ELSE gender END").alias("new_gender"))
df4.show()

# עם SQL Query
df.createOrReplaceTempView("EMP")
spark.sql("SELECT name, CASE WHEN gender = 'M' THEN 'Male' " +
          "WHEN gender = 'F' THEN 'Female' " +
          "WHEN gender IS NULL THEN '' " +
          "ELSE gender END as new_gender FROM EMP").show()

---

## 🎓 סיכום

במאמר זה, למדת כיצד להשתמש ב-PySpark SQL:
- ✅ פונקציות `when()` ו-`otherwise()` 
- ✅ ביטויי `CASE WHEN` 
- ✅ שימוש ב-DataFrame עם פונקציות כמו `withColumn()`, `select()` ו-`expr()`
- ✅ טיפול ב-NULL/None
- ✅ שימוש בתנאים מרובים עם אופרטורים לוגיים AND (`&`) ו-OR (`|`)

הפונקציות הללו חיוניות לביצוע טרנספורמציות מותנות על נתונים ב-PySpark! 🚀

**למידה מהנה!** 💪

---

## 📚 מקורות ומידע נוסף

### תיעוד רשמי:
- [Apache Spark Documentation](https://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html)

### מאמרים קשורים:
- PySpark SQL Date and Timestamp Functions
- PySpark SQL Types (DataType) with Examples  
- PySpark Get Number of Rows and Columns
- Dynamic way of doing ETL through Pyspark
- PySpark SparkContext Explained
- PySpark Shell Command Usage with Examples
- PySpark reduceByKey usage with example
- PySpark withColumn() Usage with Examples

---

### 📝 הערות:
- מחברת זו נוצרה למטרות לימוד
- מומלץ להריץ כל תא קוד בנפרד כדי להבין את התוצאות
- ניתן לשנות את הנתונים ולהתנסות עם תנאים שונים

**© SparkByExamples.com | תורגם ועובד לעברית**